# 3er Entregable

Integrantes:
- Araoz, Tania
- Bajo, Pablo
- Barrera, Manuel

## **Alternativa de modelo**

Como nos enfrentamos a una disconformidad de los indicadores con el modelo Light FM, seguimos explorando y nos encontramos con nueva alternativa. En la cuál elegímos mandarlo como un aporte extra, ya que tiene particularidad en las que no se puede utilizar en el entorno de visual studio


# Surprise

Es una libreria dedicada a la creación de sistemas de recomendación basados en filtrado colaborativo.

### Carga de datasets

In [1]:
import pandas as pd

Utilizamos el mismo, con la diferencia que lo dividimos en 500.000 datos para que sea más liviana la carga al colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
ratings = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/trabajo3/ratings500.csv")
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119
...,...,...,...,...
499993,4780,6936,3.0,1681789683
499994,4780,7293,3.0,1681790112
499995,4780,8464,4.0,1681790128
499996,4780,8528,3.5,1681789706


In [4]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499998 entries, 0 to 499997
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     499998 non-null  int64  
 1   movieId    499998 non-null  int64  
 2   rating     499998 non-null  float64
 3   timestamp  499998 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 15.3 MB


In [5]:
ratings.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

> No hay valores nulos

In [6]:
ratings['userId'].nunique()

4780

> El dataset tiene 610 ususarios. <span style="color:red">Cambiar con dataset grande</span>

In [7]:
ratings['movieId'].nunique()

18886

> el dataset contiene ratings de 9724 peliculas. <span style="color:red">Actualizar con dataset grande</span>

In [8]:
ratings['rating'].sort_values(ascending=True).unique()

array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

Nuevo Modelo

Usando surprise, SVD y Cross Validation

In [9]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162682 sha256=4c6ed6bdb02b37f67483d2f0727107b20e56cfe38fed488a491e373df3a77b41
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


Como no se necesita la columna fecha, la eliminamos

In [10]:
ratings= ratings.drop("timestamp", axis=1)

In [11]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,110,4.0
2,1,158,4.0
3,1,260,4.5
4,1,356,5.0


Creamos dos objetos necesarios para este modelo: Dataset y Reader

## Carga de librerias

In [12]:
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [14]:
from surprise import Dataset, Reader

# Objeto Reader
obj_reader = Reader(rating_scale=(ratings["rating"].min(), ratings["rating"].max()))

# Crear el dataset desde el DataFrame
dataset = Dataset.load_from_df(ratings[['userId', 'movieId','rating']], obj_reader)

dataset

Modelo y métricas

In [15]:
trainset, testset = train_test_split(dataset, test_size=0.2)

modelo = SVD(n_factors=100, n_epochs=20)

modelo.fit(trainset)

# Realizar validación cruzada y obtener métricas
cv_results = cross_validate(modelo, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Imprimir las métricas promedio
print("Métricas promedio de validación cruzada:")
print(f"RMSE: {cv_results['test_rmse'].mean()}")
print(f"MAE: {cv_results['test_mae'].mean()}")

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8618  0.8573  0.8643  0.8640  0.8614  0.8618  0.0025  
MAE (testset)     0.6555  0.6524  0.6559  0.6567  0.6534  0.6548  0.0016  
Fit time          9.05    10.19   9.35    9.93    9.10    9.52    0.46    
Test time         1.26    1.27    1.08    1.46    1.66    1.35    0.20    
Métricas promedio de validación cruzada:
RMSE: 0.8617649347203787
MAE: 0.6547616503544723


Por las métricas que nos brinda, creemos que este modelo sería el más optimo para las recomendaciones

Predicciones para un usuario específico

In [16]:
user_id = str(196)  # ID del usuario
item_id = str(308)  # ID del ítem
pred = modelo.predict(user_id, item_id)


print(f"Predicción para el usuario {user_id} y el ítem {item_id}:")
print(f"Puntuación estimada: {pred.est}")

Predicción para el usuario 196 y el ítem 308:
Puntuación estimada: 3.535965089912725


Lo que nos dice que al Usuario 196, el item 308 probablemente le guste un 3.5357

Para poder guardar las predicciones como un data, hacemos la división del data frame, ya que para el modelo se gestiono como tipo objeto

In [17]:
import pandas as pd

In [18]:
total_rows = len(ratings)

In [19]:
split_index = int(total_rows * 0.8)

In [20]:
df_train = ratings.iloc[:split_index]
df_test = ratings.iloc[split_index:]

Creamos una variable con las 20 recomendaciones para el usuario

In [21]:
def get_top_n(predictions, n=20):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
from collections import defaultdict

# Obtener las predicciones para el conjunto de prueba
predictions = modelo.test(testset)

# Obtener las mejores recomendaciones para cada usuario
top_n = get_top_n(predictions, n=20)  # Puedes ajustar el valor de 'n' según tus necesidades

# Imprimir las recomendaciones para cada usuario
for uid, user_ratings in top_n.items():
    print(f"Usuario: {uid}")
    print("Recomendaciones:")
    for iid, est in user_ratings:
        print(f"Item ID: {iid}, Estimación de calificación: {est}")
    print("\n")


Se truncaron las últimas líneas 5000 del resultado de transmisión.
Recomendaciones:
Item ID: 480, Estimación de calificación: 3.9193348187086694


Usuario: 4424
Recomendaciones:
Item ID: 4857, Estimación de calificación: 4.239532576065499
Item ID: 3791, Estimación de calificación: 3.5944940912660117
Item ID: 5945, Estimación de calificación: 2.8189757477896236


Usuario: 3169
Recomendaciones:
Item ID: 1357, Estimación de calificación: 3.7460907600178146
Item ID: 1188, Estimación de calificación: 3.7460289239717457


Usuario: 1628
Recomendaciones:
Item ID: 434, Estimación de calificación: 3.814305520741502


Usuario: 2259
Recomendaciones:
Item ID: 1246, Estimación de calificación: 3.9017716544930816
Item ID: 553, Estimación de calificación: 3.5184455051666625


Usuario: 4225
Recomendaciones:
Item ID: 110, Estimación de calificación: 3.4860404024485256
Item ID: 1407, Estimación de calificación: 2.346946255092747
Item ID: 2710, Estimación de calificación: 1.6804866277539399


Usuario: 375

Creamos una lista para almacenar las recomendaciones

In [ ]:
recommendations_list = []

# Iterar sobre el diccionario top_n y agregar las recomendaciones a la lista
for uid, user_ratings in top_n.items():
    for iid, est in user_ratings:
        recommendations_list.append({'Usuario': uid, 'Item ID': iid, 'Estimación de calificación': est})

# Convertir la lista de recomendaciones a un Data
recommendations_df = pd.DataFrame(recommendations_list)


print(recommendations_df)


       Usuario  Item ID  Estimación de calificación
0          916     1233                    4.309455
1          916      858                    4.078877
2          916     2959                    4.075174
3          916      800                    3.981910
4          916     1259                    3.976886
...        ...      ...                         ...
44981     2217     1479                    3.148410
44982     2358    51255                    3.643188
44983     4535     1717                    2.307876
44984     3876     3654                    4.332964
44985     1567   169218                    3.992375

[44986 rows x 3 columns]
